# Kratos Defense ML Models - Model Registry

This notebook trains ML models for the Kratos Intelligence Agent:
- **Program Risk Prediction** - Classify programs at risk of cost/schedule issues
- **Supplier Risk Prediction** - Classify suppliers at risk of quality/delivery issues
- **Production Forecasting** - Predict manufacturing order volume

All models are registered to Snowflake Model Registry.


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")

In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('KRATOS_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('KRATOS_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")

---
# MODEL 1: Program Risk Prediction


In [ ]:
# Get program features for risk prediction
program_df = session.sql("""
SELECT
    p.program_id,
    p.program_type,
    COALESCE(p.contract_type, 'UNKNOWN') AS contract_type,
    COALESCE(p.funded_value / NULLIF(p.total_contract_value, 0), 0.5)::FLOAT AS funded_ratio,
    COALESCE(p.costs_incurred / NULLIF(p.funded_value, 0), 0.5)::FLOAT AS cost_ratio,
    COALESCE(p.revenue_recognized / NULLIF(p.costs_incurred, 0), 0.8)::FLOAT AS revenue_cost_ratio,
    COALESCE(p.margin_percentage, 10)::FLOAT AS margin_pct,
    DATEDIFF('day', p.start_date, CURRENT_DATE())::FLOAT AS days_active,
    DATEDIFF('day', CURRENT_DATE(), COALESCE(p.planned_end_date, DATEADD('year', 1, CURRENT_DATE())))::FLOAT AS days_remaining,
    CASE WHEN p.risk_level = 'HIGH' OR COALESCE(p.costs_incurred, 0) > COALESCE(p.funded_value, 1) * 0.95 THEN 1 ELSE 0 END AS is_at_risk
FROM RAW.PROGRAMS p
WHERE p.program_status = 'ACTIVE'
""")

print(f"Program data: {program_df.count()} programs")
program_df.show(5)

In [ ]:
# Train/test split (80/20)
train_program, test_program = program_df.random_split([0.8, 0.2], seed=42)

# Drop PROGRAM_ID
train_program = train_program.drop("PROGRAM_ID")
test_program = test_program.drop("PROGRAM_ID")

print(f"Training set: {train_program.count()} rows")
print(f"Test set: {test_program.count()} rows")

# Create pipeline
program_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["PROGRAM_TYPE", "CONTRACT_TYPE"],
        output_cols=["PROGRAM_TYPE_ENC", "CONTRACT_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_AT_RISK"],
        output_cols=["RISK_PREDICTION"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
program_pipeline.fit(train_program)
print("✅ Program risk model trained")

In [ ]:
# Make predictions
program_predictions = program_pipeline.predict(test_program)

# Calculate metrics
accuracy = accuracy_score(df=program_predictions, y_true_col_names="IS_AT_RISK", y_pred_col_names="RISK_PREDICTION")
metrics = {"accuracy": round(accuracy, 4)}
print(f"Program risk model metrics: {metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=program_pipeline,
    model_name="PROGRAM_RISK_PREDICTOR",
    version_name="V1",
    comment="Predicts program cost/schedule risk using Random Forest based on financial metrics",
    metrics=metrics
)

print("✅ Program risk model registered to Model Registry as PROGRAM_RISK_PREDICTOR")

---
# MODEL 2: Supplier Risk Prediction


In [ ]:
# Get supplier features for risk prediction
supplier_df = session.sql("""
SELECT
    s.supplier_id,
    s.supplier_type,
    COALESCE(s.supplier_category, 'UNKNOWN') AS supplier_category,
    COALESCE(s.quality_rating, 0.8)::FLOAT AS quality_rating,
    COALESCE(s.delivery_rating, 0.8)::FLOAT AS delivery_rating,
    COALESCE(s.total_spend / 1000000, 0)::FLOAT AS spend_millions,
    DATEDIFF('day', COALESCE(s.first_order_date, CURRENT_DATE()), CURRENT_DATE())::FLOAT AS days_as_supplier,
    CASE WHEN s.is_small_business THEN 1 ELSE 0 END AS is_small_business,
    CASE WHEN COALESCE(s.quality_rating, 0.8) < 0.75 OR COALESCE(s.delivery_rating, 0.8) < 0.75 THEN 1 ELSE 0 END AS is_at_risk
FROM RAW.SUPPLIERS s
WHERE s.supplier_status = 'ACTIVE'
""")

print(f"Supplier data: {supplier_df.count()} suppliers")
supplier_df.show(5)

In [ ]:
# Train/test split (80/20)
train_supplier, test_supplier = supplier_df.random_split([0.8, 0.2], seed=42)

# Drop SUPPLIER_ID
train_supplier = train_supplier.drop("SUPPLIER_ID")
test_supplier = test_supplier.drop("SUPPLIER_ID")

print(f"Training set: {train_supplier.count()} rows")
print(f"Test set: {test_supplier.count()} rows")

# Create pipeline
supplier_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["SUPPLIER_TYPE", "SUPPLIER_CATEGORY"],
        output_cols=["SUPPLIER_TYPE_ENC", "SUPPLIER_CATEGORY_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_AT_RISK"],
        output_cols=["RISK_PREDICTION"],
        n_estimators=100,
        max_depth=8
    ))
])

# Train model
supplier_pipeline.fit(train_supplier)
print("✅ Supplier risk model trained")

In [ ]:
# Make predictions
supplier_predictions = supplier_pipeline.predict(test_supplier)

# Calculate metrics
accuracy = accuracy_score(df=supplier_predictions, y_true_col_names="IS_AT_RISK", y_pred_col_names="RISK_PREDICTION")
supplier_metrics = {"accuracy": round(accuracy, 4)}
print(f"Supplier risk model metrics: {supplier_metrics}")

# Register model
reg.log_model(
    model=supplier_pipeline,
    model_name="SUPPLIER_RISK_PREDICTOR",
    version_name="V1",
    comment="Predicts supplier quality/delivery risk using Random Forest based on performance ratings",
    metrics=supplier_metrics
)

print("✅ Supplier risk model registered to Model Registry as SUPPLIER_RISK_PREDICTOR")

---
# MODEL 3: Production Forecasting


In [ ]:
# Get monthly production data with features
production_df = session.sql("""
SELECT
    DATE_TRUNC('month', order_date)::DATE AS order_month,
    MONTH(order_date)::FLOAT AS month_num,
    YEAR(order_date)::FLOAT AS year_num,
    COUNT(DISTINCT order_id)::FLOAT AS order_count,
    COALESCE(SUM(quantity_ordered), 0)::FLOAT AS total_quantity,
    COALESCE(SUM(total_cost), 0)::FLOAT AS total_cost
FROM RAW.MANUFACTURING_ORDERS
GROUP BY DATE_TRUNC('month', order_date), MONTH(order_date), YEAR(order_date)
HAVING COUNT(DISTINCT order_id) > 0
ORDER BY order_month
""")

print(f"Production data: {production_df.count()} months")
production_df.show(5)

In [ ]:
# Use 80/20 split
train_production, test_production = production_df.random_split([0.8, 0.2], seed=42)

# Drop ORDER_MONTH (DATE type not supported in pipeline)
train_production = train_production.drop("ORDER_MONTH")
test_production = test_production.drop("ORDER_MONTH")

print(f"Training set: {train_production.count()} rows")
print(f"Test set: {test_production.count()} rows")

# Create pipeline
production_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "TOTAL_QUANTITY", "TOTAL_COST"],
        output_cols=["MONTH_NUM_SCALED", "TOTAL_QUANTITY_SCALED", "TOTAL_COST_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["ORDER_COUNT"],
        output_cols=["PREDICTED_ORDERS"]
    ))
])

# Train model
production_pipeline.fit(train_production)
print("✅ Production forecast model trained")

In [ ]:
# Make predictions on test set
production_predictions = production_pipeline.predict(test_production)

# Calculate metrics
mae = mean_absolute_error(df=production_predictions, y_true_col_names="ORDER_COUNT", y_pred_col_names="PREDICTED_ORDERS")
mse = mean_squared_error(df=production_predictions, y_true_col_names="ORDER_COUNT", y_pred_col_names="PREDICTED_ORDERS")
rmse = mse ** 0.5

production_metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Production forecast metrics: {production_metrics}")

# Register model
reg.log_model(
    model=production_pipeline,
    model_name="PRODUCTION_FORECASTER",
    version_name="V1",
    comment="Forecasts monthly manufacturing order count using Linear Regression",
    metrics=production_metrics
)

print("✅ Production forecast model registered to Model Registry as PRODUCTION_FORECASTER")

---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
print(reg.show_models())

# Show versions for each model
print("\nProgram Risk model versions:")
print(reg.get_model("PROGRAM_RISK_PREDICTOR").show_versions())

print("\nSupplier Risk model versions:")
print(reg.get_model("SUPPLIER_RISK_PREDICTOR").show_versions())

print("\nProduction Forecaster versions:")
print(reg.get_model("PRODUCTION_FORECASTER").show_versions())

print("\n✅ All models registered and ready to add to Intelligence Agent")